In [474]:
import pandas as pd
import numpy as np
import pickle

### Scrape the First Page of the Coffee Data

In [475]:
import requests
from bs4 import BeautifulSoup
source_code = requests.get('https://database.coffeeinstitute.org/coffees/arabica')
soup = BeautifulSoup(source_code.text, 'html5lib')

In [514]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
chromedriver = "/Applications/chromedriver" 
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

### Grab the entire list of coffees from 1st page

In [459]:
driver.get('https://database.coffeeinstitute.org/coffees/arabica')
all_coffee=pd.DataFrame()

for page in range(1,28):
    soup = BeautifulSoup(driver.page_source, 'html5lib')
    time.sleep(3)
    
    one_list=soup.find('table', {'data-table':'coffees'})
    time.sleep(3)    
    
    one_list = pd.read_html(driver.page_source)
    time.sleep(3)
    
    one_list = pd.DataFrame(one_list[0])
    one_list.drop(['Unnamed: 0'], axis = 1, inplace = True)
    one_list['ID']=one_list['ID'].str.strip('#')
    all_coffee=pd.concat([all_coffee, one_list], axis=0)
    
    next_button = driver.find_element_by_xpath("//a[contains(text(), 'Next')]")
    next_button.click()

print(all_coffee.shape)
all_coffee.sort_values(by=["ID"], inplace=True)
all_coffee[:10]

(1350, 7)


,ID,Species,Country,Owner,Grade,ICP,Completed
34,100409,Arabica,Mexico,ALFREDO BOJALIL,79.67,amecafe,"June 4th, 2012"
30,10044,Arabica,Nicaragua,"Exportadora Atlantic, S.A.",80.17,acen,"April 24th, 2015"
26,102275,Arabica,"Tanzania, United Republic Of",DAE Ltd Company,81.67,afca,"January 2nd, 2015"
20,103581,Arabica,Mexico,CHRISTINA DUSING,81.67,amecafe,"July 27th, 2012"
49,103804,Arabica,Taiwan,CQI Taiwan ICP CQI台灣合作夥伴,80.42,bvi,"December 26th, 2013"
6,105651,Arabica,Mexico,MIGUEL CORTES MORENO,80.33,amecafe,"August 30th, 2012"
4,107500,Arabica,Brazil,Ipanema Coffees,83.08,bsca,"October 8th, 2015"
37,107587,Arabica,Mexico,OBED RENDON PONCE,79.58,amecafe,"September 17th, 2012"
0,108866,Arabica,Honduras,Elsy Reyes,82.50,ihcafe,"June 3rd, 2017"
23,10892,Arabica,Mexico,ORGANIZACIONES DE PRODUCTORES DE CAFE COLIMENSE,81.00,amecafe,"July 3rd, 2012"


In [501]:
# Resets the index
ind=range(len(all_coffee))
all_coffee['ind']=ind
all_coffee.set_index(['ind'], inplace=True)
all_coffee.sort_index(inplace=True)
all_coffee.head()

,ID,Species,Country,Owner,Grade,ICP,Completed
ind,,,,,,,
0,100409,Arabica,Mexico,ALFREDO BOJALIL,79.67,amecafe,"June 4th, 2012"
1,10044,Arabica,Nicaragua,"Exportadora Atlantic, S.A.",80.17,acen,"April 24th, 2015"
2,102275,Arabica,"Tanzania, United Republic Of",DAE Ltd Company,81.67,afca,"January 2nd, 2015"
3,103581,Arabica,Mexico,CHRISTINA DUSING,81.67,amecafe,"July 27th, 2012"
4,103804,Arabica,Taiwan,CQI Taiwan ICP CQI台灣合作夥伴,80.42,bvi,"December 26th, 2013"


__Pickle this data__

In [460]:
# os.chdir('/Users/bellepeng/Desktop/Metis/Work/Projects/')
!pwd

/Users/bellepeng/Desktop/Metis/Work/Projects


In [502]:
with open('all_coffee.pkl', 'wb') as picklefile:
    pickle.dump(all_coffee, picklefile)

In [504]:
with open("all_coffee.pkl", 'rb') as picklefile: 
    all_coffee = pickle.load(picklefile)

all_coffee.head()

,ID,Species,Country,Owner,Grade,ICP,Completed
ind,,,,,,,
0,100409,Arabica,Mexico,ALFREDO BOJALIL,79.67,amecafe,"June 4th, 2012"
1,10044,Arabica,Nicaragua,"Exportadora Atlantic, S.A.",80.17,acen,"April 24th, 2015"
2,102275,Arabica,"Tanzania, United Republic Of",DAE Ltd Company,81.67,afca,"January 2nd, 2015"
3,103581,Arabica,Mexico,CHRISTINA DUSING,81.67,amecafe,"July 27th, 2012"
4,103804,Arabica,Taiwan,CQI Taiwan ICP CQI台灣合作夥伴,80.42,bvi,"December 26th, 2013"


### Grab the next level of pages by looping over the list of `all_coffee`

In [508]:
def get_one_coffee(i, one_coffee):
    # Table 1 - Total Score
    d = {'ID': [all_coffee['ID'][i]], 'Total_Score': [float(one_coffee[0][0][0])]}
    Total_Score = pd.DataFrame(data=d)
    Total_Score
    
    # Table 2 - Info
    info1=one_coffee[1][[0, 1]]
    info2=one_coffee[1][[2, 3]]
    info2.rename(columns={2:0, 3:1}, inplace=True)
    info0=pd.concat([info1, info2],  axis=0)
    info=info0.set_index(0).T
    info['ID']=all_coffee['ID'][i]

    # Table 3 - Cupping
    cupping1=one_coffee[2][[0, 1]]
    cupping2=one_coffee[2][[2, 3]]
    cupping2.rename(columns={2:0, 3:1}, inplace=True)
    cupping0=pd.concat([cupping1, cupping2],  axis=0)
    cupping0=cupping0.loc[cupping0[0] .isin(["Acidity", "Sweetness"])]
    cupping=cupping0.set_index(0).T
    cupping['ID']=all_coffee['ID'][i]
    
    # Table 4 - Green
    green1=one_coffee[3][[0, 1]]
    green2=one_coffee[3][[2, 3]]
    green2.rename(columns={2:0, 3:1}, inplace=True)
    green0=pd.concat([green1, green2],  axis=0).set_index(0)
    green0.drop([None], inplace=True)
    green=green0.T
    green['ID']=all_coffee['ID'][i]
    
    merge1=pd.merge(Total_Score, info, how="outer", on="ID")
    merge2=pd.merge(merge1, cupping, how="outer", on="ID")
    merge3=pd.merge(merge2, green, how="outer", on="ID")
    return merge3

In [510]:
len(all_coffee)

1350

In [515]:
coffee_master=pd.DataFrame()

for i in range(50):
    # Get the soup for one coffee
    url='https://database.coffeeinstitute.org/coffee/'+all_coffee['ID'][i]
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html5lib')
    one_coffee=pd.read_html(driver.page_source)
    
    # parse the data
    coffee1=get_one_coffee(i, one_coffee)
    
    # Add to master dataframe
    coffee_master=pd.concat([coffee_master, coffee1], axis=0)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [ ]:
print(coffee_master.shape)
coffee_master.sort_values(by=["ID"], inplace=True)
coffee_master[:10]

In [ ]:
with open('coffee_master.pkl', 'wb') as picklefile:
    pickle.dump(coffee_master, picklefile)

In [ ]:
with open("coffee_master.pkl", 'rb') as picklefile: 
    coffee_master = pickle.load(picklefile)

coffee_master.head()

__Testing 1 page : Delete if above code is working__

In [363]:
driver.get('https://database.coffeeinstitute.org/coffee/512197')
soup = BeautifulSoup(driver.page_source, 'html5lib')
time.sleep(1)
one_coffee=pd.read_html(driver.page_source)
len(one_coffee)

5

In [346]:
# Table 1 - I only want the total score
d = {'ID': [all_coffee['ID'][0]], 'Total_Score': [float(one_coffee[0][0][0])]}
Total_Score = pd.DataFrame(data=d)
Total_Score

,ID,Total_Score
0,512197,90.58


In [337]:
# Table 2
info1=one_coffee[1][[0, 1]]
info2=one_coffee[1][[2, 3]]
info2.rename(columns={2:0, 3:1}, inplace=True)
info0=pd.concat([info1, info2],  axis=0)
info=info0.set_index(0).T
info['ID']=all_coffee['ID'][0]
# info.set_index('ID', inplace=True)
info

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,Country of Origin,Farm Name,Lot Number,Mill,ICO Number,Company,Altitude,Region,Producer,Number of Bags,Bag Weight,In-Country Partner,Harvest Year,Grading Date,Owner,Variety,Status,Processing Method,ID
1,Ethiopia,METAD PLC,NaN,METAD PLC,2014/2015,METAD Agricultural Developmet plc,1950-2200,GUJI-HAMBELA/GOYO,METAD PLC,300,60 kg,METAD Agricultural Development plc,2014,"April 4th, 2015",metad plc,NaN,Completed,Washed / Wet,512197


In [338]:
# Table 3
cupping1=one_coffee[2][[0, 1]]
cupping2=one_coffee[2][[2, 3]]
cupping2.rename(columns={2:0, 3:1}, inplace=True)
cupping0=pd.concat([cupping1, cupping2],  axis=0)
cupping0
cupping0=cupping0.loc[cupping0[0] .isin(["Acidity", "Sweetness"])]
cupping=cupping0.set_index(0).T
cupping['ID']=all_coffee['ID'][0]
# cupping.set_index('ID', inplace=True)
cupping

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,Acidity,Sweetness,ID
1,8.75,10.00,512197


In [339]:
# Table 4
green1=one_coffee[3][[0, 1]]
green2=one_coffee[3][[2, 3]]
green2.rename(columns={2:0, 3:1}, inplace=True)
green0=pd.concat([green1, green2],  axis=0).set_index(0)
green0.drop([None], inplace=True)
green=green0.T
green['ID']=all_coffee['ID'][0]
# green.set_index('ID', inplace=True)
green

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,Moisture,Category One Defects,Quakers,Color,Category Two Defects,ID
1,12 %,0 full defects,0,Green,0 full defects,512197


In [351]:
# Put everything together
# Total_Score
# info
# cupping
# green
merge1=pd.merge(Total_Score, info, how="outer", on="ID")
merge2=pd.merge(merge1, cupping, how="outer", on="ID")
merge3=pd.merge(merge2, green, how="outer", on="ID")
merge3

,ID,Total_Score,Country of Origin,Farm Name,Lot Number,Mill,ICO Number,Company,Altitude,Region,...,Variety,Status,Processing Method,Acidity,Sweetness,Moisture,Category One Defects,Quakers,Color,Category Two Defects
0,512197,90.58,Ethiopia,METAD PLC,NaN,METAD PLC,2014/2015,METAD Agricultural Developmet plc,1950-2200,GUJI-HAMBELA/GOYO,...,NaN,Completed,Washed / Wet,8.75,10.00,12 %,0 full defects,0,Green,0 full defects


In [382]:
try: 
    x=1+'cat'
except(TypeError,ConnectionResetError):
    print('don\'t do that!')

don't do that!
